# Context
This notebook drives the training process for different models.

In [1]:
# Set project's environment variables
import os
import sys
from dotenv import load_dotenv
load_dotenv(dotenv_path="../project.env")
sys.path.append(os.environ["PYTHONPATH"])

In [ ]:
# Import project-wide and PH2 specific variables and functions
import superheader as sup
import TRAINheader as train

# Dataset

In [ ]:
sup.report_dir_if_not_exists(sup.PH3_DATA_ROOT)

# Model training

## KNN

### Record keeping

In [4]:
best_scores = {
    "active_hand": {
        "score": -1,
        "data_unit": "",
        "subcode": "",
        "sub2code": "",
        "reducer": "",
        "kernel": "",
        "n": -1,
        "k": -1
    },
    "class_numeric": {
        "score": -1,
        "data_unit": "",
        "subcode": "",
        "sub2code": "",
        "reducer": "",
        "kernel": "",
        "n": -1,
        "k": -1
    }
}

def update_best_knn(score, label_col, data_unit, subcode, sub2code, reducer_name, kernel_name, n, k):
    key = (
        "active_hand" if label_col == sup.active_hand_col
        else "class_numeric" if label_col == sup.class_numeric_column
        else None
    )
    if key and score > best_scores[key]["score"]:
        print("updating best...")
        best_scores[key].update({
            "score": score,
            "data_unit": data_unit,
            "subcode": subcode,
            "sub2code": sub2code,
            "reducer": reducer_name,
            "kernel": kernel_name,
            "n": n,
            "k": k
        })

def print_best_knn(label_col):
    key = (
        "active_hand" if label_col == sup.active_hand_col
        else "class_numeric" if label_col == sup.class_numeric_column
        else None
    )
    if key:
        best = best_scores[key]
        print(f"Best score: {best['score']}")
        print(f"Best k: {best['k']}")
        print(f"Best n: {best['n']}")
        print(f"Data unit: {best['data_unit']}")
        print(f"PH2 or no PH2: {best['subcode']}")
        print(f"Reduced or scaled: {best['sub2code']}")
        print(f"Best reducer: {best['reducer']}")
        print(f"Best kernel: {best['kernel']}")


### Helper functions

In [5]:
def try_all_n_k(data_unit, label_col, subcode, sub2code, reducer_name="", n="", kernel_name=""):
  for k in train.TRAIN_KNN_K_CANDIDATES:
    data_path = os.path.join(sup.PH3_DATA_ROOT, subcode, sub2code, reducer_name, kernel_name, f"{data_unit}{n}.csv")
    model = train.arch.KNN(data_path=data_path, label_col=label_col, k=k)
    model.fit()
    score = model.score()
    print(f"n={n}; k={k}; score: {score}")

    update_best_knn(score, label_col, data_unit, subcode, sub2code, reducer_name, kernel_name, n, k)

In [ ]:
def best_KNN(data_unit, label_col):
  for subcode in sup.PH3_SUB_CODES:
    for sub2code in sup.PH3_SUB2_CODES:
      if sub2code != sup.PH3_REDUCED_CODE:
        try_all_n_k(data_unit, label_col, subcode, sub2code)
      else:
        for reducer_name in sup.PH3_REDUCER_NAMES:
          for n in sup.PH3_N_CANDIDATES:
            if reducer_name != sup.PH3_REDUCER_NAME_KPCA:
              try_all_n_k(data_unit, label_col, subcode, sub2code, reducer_name, n)
            else:
              for kernel_name in sup.PH3_REDUCER_KERNEL_NAMES:
                try_all_n_k(data_unit, label_col, subcode, sub2code, reducer_name, n, kernel_name)
  
  print_best_knn(label_col)
  

### Active Hand prediction

In [ ]:
best_KNN("perframe", sup.active_hand_col)

### Sign prediction

In [ ]:
best_KNN("perframe", sup.class_numeric_column)

In [ ]:
best_KNN("pervideo", sup.class_numeric_column)